In [114]:
import json
import pandas as pd
from lxml import etree as etree
import re
import ast

#  SQL queries

The textes where prepared using SQL queries – exported in CSV format with '#' separator

Internal note : the SQL queries are stored a specific SQL file (AMPI_texts)

# Texts

In [2]:
texts_f = 'texte/all_texts_20220323.csv'

In [3]:
texts = pd.read_csv(texts_f, sep='#')
print(len(texts))
texts.head()

783


,id_manuscript,id_expression,title_manuscript,id_section,title_section,fk_type,max,id_digital,text_version,id_text,quill_doc
0,835549,835550,Heft 1 (1795),835573,23.11.1795,833900.0,"Section – Name: '23.11.1795', is part of: 'StA...",24651936,10,23381914,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
1,835549,835550,Heft 1 (1795),835641,27.11.1795,833900.0,"Section – Name: '27.11.1795', is part of: 'StA...",24651950,5,23381935,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
2,835549,835550,Heft 1 (1795),835651,28.11.1795,833900.0,"Section – Name: '28.11.1795', is part of: 'StA...",24651951,8,23381937,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
3,835549,835550,Heft 1 (1795),835662,10.12.1795,833900.0,"Section – Name: '10.12.1795', is part of: 'StA...",24651952,6,23381939,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
4,835617,835618,Heft 2 (1796),835671,XX.XX.1796,833900.0,"Section – Name: 'XX.XX.1796', is part of: 'StA...",24651953,6,23381941,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."


In [4]:
a = texts[texts['id_manuscript'] == 884917]
print(len(a))
a

18


,id_manuscript,id_expression,title_manuscript,id_section,title_section,fk_type,max,id_digital,text_version,id_text,quill_doc
141,884917,884918,Heft 12 (1815-1816),884929,14.05.1815,833900.0,"Section – Name: '14.05.1815', is part of: 'StA...",24668641,5,23399767,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
142,884917,884918,Heft 12 (1815-1816),884937,15.05.1815,833900.0,"Section – Name: '15.05.1815', is part of: 'StA...",24668642,3,23399768,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
143,884917,884918,Heft 12 (1815-1816),884945,21.05.1816,833900.0,"Section – Name: '21.05.1816', is part of: 'StA...",24668643,6,23399769,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
144,884917,884918,Heft 12 (1815-1816),884953,22.05.1815,833900.0,"Section – Name: '22.05.1815', is part of: 'StA...",24668644,7,23399770,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
145,884917,884918,Heft 12 (1815-1816),884961,XX.XX.1815,833900.0,"Section – Name: 'XX.XX.1815', is part of: 'StA...",24668645,13,23399771,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
146,884917,884918,Heft 12 (1815-1816),884961,XX.XX.1815,833900.0,"Section – Name: 'XX.XX.1815', is part of: 'StA...",24668699,5,23399857,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
147,884917,884918,Heft 12 (1815-1816),884961,XX.XX.1815,833900.0,"Section – Name: 'XX.XX.1815', is part of: 'StA...",24668700,5,23399858,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
148,884917,884918,Heft 12 (1815-1816),884968,17.10.1815,833900.0,"Section – Name: '17.10.1815', is part of: 'StA...",24668646,6,23399772,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
149,884917,884918,Heft 12 (1815-1816),884976,22.10.1815,833900.0,"Section – Name: '22.10.1815', is part of: 'StA...",24668647,3,23399773,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."
150,884917,884918,Heft 12 (1815-1816),884984,30.10.1815,833900.0,"Section – Name: '30.10.1815', is part of: 'StA...",24668648,2,23399774,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari..."


In [5]:
def f_split (a):    
    try:
        out = [e.strip() for e in a.split(',') if 'has type' in e][0].split(':')[1]
    except:
        out = ''
    return out
    

In [6]:
texts['section_type'] = texts['max'].apply(lambda x : f_split(x))

In [7]:
texts.head()

,id_manuscript,id_expression,title_manuscript,id_section,title_section,fk_type,max,id_digital,text_version,id_text,quill_doc,section_type
0,835549,835550,Heft 1 (1795),835573,23.11.1795,833900.0,"Section – Name: '23.11.1795', is part of: 'StA...",24651936,10,23381914,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'
1,835549,835550,Heft 1 (1795),835641,27.11.1795,833900.0,"Section – Name: '27.11.1795', is part of: 'StA...",24651950,5,23381935,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'
2,835549,835550,Heft 1 (1795),835651,28.11.1795,833900.0,"Section – Name: '28.11.1795', is part of: 'StA...",24651951,8,23381937,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'
3,835549,835550,Heft 1 (1795),835662,10.12.1795,833900.0,"Section – Name: '10.12.1795', is part of: 'StA...",24651952,6,23381939,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'
4,835617,835618,Heft 2 (1796),835671,XX.XX.1796,833900.0,"Section – Name: 'XX.XX.1796', is part of: 'StA...",24651953,6,23381941,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'


## Analyse textes

In [8]:
### Effectifs sections-textes par cahier
ts = texts.groupby('id_manuscript').size()   ##.count()
print(len(ts))
ts

32


id_manuscript
835549     4
835617     8
870955     1
871000     9
871143    19
872612     1
872632     6
872737     4
872828     5
872911    19
874181    65
884917    18
891344    27
891867    23
892261    14
897086    28
897351    30
897613    31
897893     3
897927    31
898222    28
898484    25
906997    45
907420    30
907701    26
907946    33
908255    36
908596    49
908607    49
908618    44
908628    35
908640    37
dtype: int64

In [9]:
### Liste des manuscrits (= cahiers, 'Hefte') —  id_ms : 884917
list(ts.index)

[835549,
 835617,
 870955,
 871000,
 871143,
 872612,
 872632,
 872737,
 872828,
 872911,
 874181,
 884917,
 891344,
 891867,
 892261,
 897086,
 897351,
 897613,
 897893,
 897927,
 898222,
 898484,
 906997,
 907420,
 907701,
 907946,
 908255,
 908596,
 908607,
 908618,
 908628,
 908640]

### Dictionnaire des textes : texts_li

In [98]:
texts_li = texts.to_dict(orient='records')
texts_li[:1]

[{'id_manuscript': 835549,
  'id_expression': 835550,
  'title_manuscript': 'Heft 1 (1795)',
  'id_section': 835573,
  'title_section': '23.11.1795',
  'fk_type': 833900.0,
  'max': "Section – Name: '23.11.1795', is part of: 'StABS PA 511a 304-03-04', has type: 'Tagebucheintrag', was created by: '23.11.1795'",
  'id_digital': 24651936,
  'text_version': 10,
  'id_text': 23381914,
  'quill_doc': '{"ops": [{"insert": "[", "attributes": {"charid": "2767"}}, {"insert": "2", "attributes": {"charid": "20"}}, {"insert": "3", "attributes": {"charid": "21"}}, {"insert": ".", "attributes": {"charid": "22"}}, {"insert": " ", "attributes": {"charid": "23"}}, {"insert": "N", "attributes": {"charid": "24"}}, {"insert": "o", "attributes": {"charid": "25"}}, {"insert": "v", "attributes": {"charid": "26"}}, {"insert": "e", "attributes": {"charid": "27"}}, {"insert": "m", "attributes": {"charid": "28"}}, {"insert": "b", "attributes": {"charid": "29"}}, {"insert": "e", "attributes": {"charid": "30"}}, {"

In [121]:
test_text = ast.literal_eval([t for t in texts_li if t['id_digital']== 24668642][0]['quill_doc'])['ops']

In [128]:
[(e, test_text[test_text.index(e)+1]) for e  in test_text if e['insert'] == '-']

[({'insert': '-', 'attributes': {'charid': '2887'}},
  {'insert': '\n', 'attributes': {'blockid': '2888'}}),
 ({'insert': '-', 'attributes': {'charid': '4132'}},
  {'insert': '\n', 'attributes': {'blockid': '4133'}})]

In [11]:
texts_li[-72:-70]

[{'id_manuscript': 908628,
  'id_expression': 908630,
  'title_manuscript': 'Heft 31 (1833-1834)',
  'id_section': 910276,
  'title_section': '02.09.1833 (?)',
  'fk_type': 833900.0,
  'max': "Section – Name: '02.09.1833 (?)', is part of: 'StABS PA 511 304-03-04', has type: 'Tagebucheintrag', was created by: '02.09.1833 (?)'",
  'id_digital': 24679598,
  'text_version': 3,
  'id_text': 23413201,
  'quill_doc': '{"ops": [{"insert": "[", "attributes": {"charid": "10973"}}, {"insert": "0", "attributes": {"charid": "10974"}}, {"insert": "2", "attributes": {"charid": "10975"}}, {"insert": ".", "attributes": {"charid": "10976"}}, {"insert": " ", "attributes": {"charid": "10977"}}, {"insert": "S", "attributes": {"charid": "10978"}}, {"insert": "e", "attributes": {"charid": "10979"}}, {"insert": "p", "attributes": {"charid": "10980"}}, {"insert": "t", "attributes": {"charid": "10981"}}, {"insert": "e", "attributes": {"charid": "10982"}}, {"insert": "m", "attributes": {"charid": "10983"}}, {"in

## Dictionnaire des cahiers : hefte_li

In [66]:
hefte_li = texts[['id_manuscript','title_manuscript', 'id_expression']].drop_duplicates().to_dict(orient='records')
hefte_li

[{'id_manuscript': 835549,
  'title_manuscript': 'Heft 1 (1795)',
  'id_expression': 835550},
 {'id_manuscript': 835617,
  'title_manuscript': 'Heft 2 (1796)',
  'id_expression': 835618},
 {'id_manuscript': 870955,
  'title_manuscript': 'Heft 3 (1796)',
  'id_expression': 870956},
 {'id_manuscript': 871000,
  'title_manuscript': 'Heft 4 (1796)',
  'id_expression': 871001},
 {'id_manuscript': 871143,
  'title_manuscript': 'Heft 5 (1797)',
  'id_expression': 871145},
 {'id_manuscript': 872612,
  'title_manuscript': 'Heft 6 (1797)',
  'id_expression': 872613},
 {'id_manuscript': 872632,
  'title_manuscript': 'Heft 7 (1798)',
  'id_expression': 872634},
 {'id_manuscript': 872737,
  'title_manuscript': 'Heft 8 (1798)',
  'id_expression': 872738},
 {'id_manuscript': 872828,
  'title_manuscript': 'Heft 9 (1801)',
  'id_expression': 872829},
 {'id_manuscript': 872911,
  'title_manuscript': 'Heft 10 (1807-1810)',
  'id_expression': 872914},
 {'id_manuscript': 874181,
  'title_manuscript': 'Heft

### Doublons

Parfois il y a deux textes pour la même section

In [13]:
texts[texts['id_section'] == 871027]

,id_manuscript,id_expression,title_manuscript,id_section,title_section,fk_type,max,id_digital,text_version,id_text,quill_doc,section_type
14,871000,871001,Heft 4 (1796),871027,15.02.1797,833900.0,"Section – Name: '15.02.1797', is part of: 'StA...",24657454,3,23387970,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'
15,871000,871001,Heft 4 (1796),871027,15.02.1797,833900.0,"Section – Name: '15.02.1797', is part of: 'StA...",24657457,3,23387973,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'


In [14]:
### Effectifs sections-textes par cahier
ts = texts.groupby('id_section').size()   ##.count()
print(len(ts))
s = ts[ts > 1]
s

775


id_section
835721    2
871027    2
884961    3
891460    2
891508    2
892343    2
897791    2
dtype: int64

In [15]:
texts[texts['id_expression'] == 884918][:3]

,id_manuscript,id_expression,title_manuscript,id_section,title_section,fk_type,max,id_digital,text_version,id_text,quill_doc,section_type
141,884917,884918,Heft 12 (1815-1816),884929,14.05.1815,833900.0,"Section – Name: '14.05.1815', is part of: 'StA...",24668641,5,23399767,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'
142,884917,884918,Heft 12 (1815-1816),884937,15.05.1815,833900.0,"Section – Name: '15.05.1815', is part of: 'StA...",24668642,3,23399768,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'
143,884917,884918,Heft 12 (1815-1816),884945,21.05.1816,833900.0,"Section – Name: '21.05.1816', is part of: 'StA...",24668643,6,23399769,"{""ops"": [{""insert"": ""["", ""attributes"": {""chari...",'Tagebucheintrag'


## Exemple de texte

In [16]:
data = json.loads(texts_li[0]['quill_doc'])
print(type(data), data)

<class 'dict'> {'ops': [{'insert': '[', 'attributes': {'charid': '2767'}}, {'insert': '2', 'attributes': {'charid': '20'}}, {'insert': '3', 'attributes': {'charid': '21'}}, {'insert': '.', 'attributes': {'charid': '22'}}, {'insert': ' ', 'attributes': {'charid': '23'}}, {'insert': 'N', 'attributes': {'charid': '24'}}, {'insert': 'o', 'attributes': {'charid': '25'}}, {'insert': 'v', 'attributes': {'charid': '26'}}, {'insert': 'e', 'attributes': {'charid': '27'}}, {'insert': 'm', 'attributes': {'charid': '28'}}, {'insert': 'b', 'attributes': {'charid': '29'}}, {'insert': 'e', 'attributes': {'charid': '30'}}, {'insert': 'r', 'attributes': {'charid': '31'}}, {'insert': ' ', 'attributes': {'charid': '32'}}, {'insert': '1', 'attributes': {'charid': '33'}}, {'insert': '7', 'attributes': {'charid': '34'}}, {'insert': '9', 'attributes': {'charid': '35'}}, {'insert': '5', 'attributes': {'charid': '36'}}, {'insert': ']', 'attributes': {'charid': '2765'}}, {'insert': '\n', 'attributes': {'blockid'

In [17]:
insert_l = data['ops']
insert_l[:10]

[{'insert': '[', 'attributes': {'charid': '2767'}},
 {'insert': '2', 'attributes': {'charid': '20'}},
 {'insert': '3', 'attributes': {'charid': '21'}},
 {'insert': '.', 'attributes': {'charid': '22'}},
 {'insert': ' ', 'attributes': {'charid': '23'}},
 {'insert': 'N', 'attributes': {'charid': '24'}},
 {'insert': 'o', 'attributes': {'charid': '25'}},
 {'insert': 'v', 'attributes': {'charid': '26'}},
 {'insert': 'e', 'attributes': {'charid': '27'}},
 {'insert': 'm', 'attributes': {'charid': '28'}}]

In [18]:
type(insert_l)

list

In [19]:
print(''.join([l['insert'] for l in insert_l]))

[23. November 1795]

[3]

95 11 23
Wer mir die für mich bittere Wahrheit 
sagt, ohne den geringsten Eigenuz dabei zu 
haben, der muß Achtung und Freundschaft für mich 
haben, und ich kan ihm die meinige, ohne Un-
gerechtigkeit nicht versagen. Ich wäre der 
Achtung nicht werth wen ich diese Wahrheiten 
nicht scharf zu Herzen nehme und nicht trachtete 
die mir vorgeworfenen Fehler an mir zu 
bessern. Es wird schwer halten, obschon ich 
den besten Willen dazu habe.
Ich hätte einen allzu strengen sittlichen Stolz 
der mich, bei der geringsten Bemerkung eines 
Fehlers an Andern, hindere, das Gute zu bemerken, 
das an ihnen ist, und mich von den 
Menschen zurückzöge, und in Fall setze ungerecht gegen sie
zu sein. Es mag vieles daran wahr sein, 
doch glaube ich daß auch vieles Misverstand dabei 
einlaufe.
Der Anlas war gewiß mein Betragen gegen S. aber 
gewiß ich schätz diesen Menschen finde viel edles 
an ihm. Die Ursache warum ich mich <und die Meinigen> von ihm so viel möglich entferne, si

# Chunks

In [20]:
chunks_f = 'texte/all_chunks_20220323.csv'

In [21]:
chunks = pd.read_csv(chunks_f, sep='|')

print(len(chunks))
chunks.head()

4316


,id_manuscript,id_section,id_text,id_entity,entity_label,fk_class,class_label,pk_entity,schema_name,table_name,...,metadata,pk_text,quill_doc,string,fk_text,fk_entity_version,id_for_import,id_for_import_txt,fk_publication_status,fk_license
0,835550,835573,23381914,873964,Selbstkritik,635,Tag,24659470,data,chunk,...,NaN,23390137,"{""ops"": [{""insert"": ""F"", ""attributes"": {""chari...",Fehler an mir zu \nbessern,23381914,6,NaN,NaN,NaN,NaN
1,835550,835573,23381914,873957,Freundschaft,635,Tag,24659471,data,chunk,...,NaN,23390138,"{""ops"": [{""insert"": ""F"", ""attributes"": {""chari...",Freundschaft,23381914,6,NaN,NaN,NaN,NaN
2,835550,835573,23381914,873838,Unbekannte Person,635,Tag,24679027,data,chunk,...,NaN,23411801,"{""ops"": [{""insert"": ""S"", ""attributes"": {""chari...",S.,23381914,8,NaN,NaN,NaN,NaN
3,835550,835573,23381914,874090,Personenbeschreibung,635,Tag,24679028,data,chunk,...,NaN,23411802,"{""ops"": [{""insert"": ""s"", ""attributes"": {""chari...",seine Herkunft,23381914,9,NaN,NaN,NaN,NaN
4,835550,835573,23381914,874226,Gedanken zur Gesellschaft,635,Tag,24679029,data,chunk,...,NaN,23411803,"{""ops"": [{""insert"": ""S"", ""attributes"": {""chari...",Stand,23381914,9,NaN,NaN,NaN,NaN


In [22]:
chunks.tail()

,id_manuscript,id_section,id_text,id_entity,entity_label,fk_class,class_label,pk_entity,schema_name,table_name,...,metadata,pk_text,quill_doc,string,fk_text,fk_entity_version,id_for_import,id_for_import_txt,fk_publication_status,fk_license
4311,908642,908930,23412704,868665,Emanuel Hoffmann-Preiswerk,21,Person,24684610,data,chunk,...,NaN,23419531,"{""ops"": [{""insert"": ""E"", ""attributes"": {""chari...",Eltern,23412704,2,NaN,NaN,NaN,NaN
4312,908642,908930,23412704,899058,Alter(n) anderer Personen,635,Tag,24684611,data,chunk,...,NaN,23419532,"{""ops"": [{""insert"": ""v"", ""attributes"": {""chari...",vorgerückten Jahren,23412704,2,NaN,NaN,NaN,NaN
4313,908642,908937,23412705,873950,Ereignis,635,Tag,24684612,data,chunk,...,NaN,23419533,"{""ops"": [{""insert"": ""v"", ""attributes"": {""chari...",verlobte,23412705,3,NaN,NaN,NaN,NaN
4314,908642,908937,23412705,867962,Rudolf Paravicini-Vischer,21,Person,24684613,data,chunk,...,NaN,23419534,"{""ops"": [{""insert"": ""R"", ""attributes"": {""chari...",R. Para.,23412705,3,NaN,NaN,NaN,NaN
4315,908642,908937,23412705,868701,Juliane Paravicini-Vischer,21,Person,24684614,data,chunk,...,NaN,23419535,"{""ops"": [{""insert"": "" "", ""attributes"": {""chari...",J. V.,23412705,3,NaN,NaN,NaN,NaN


In [62]:
### Noter qu'on nomme à tort ici id_manuscrit ce qui est en réalité le id_expression !!! À corriger dans la requête
chunks[chunks['id_manuscript']== 884918][:3]

,id_manuscript,id_section,id_text,id_entity,entity_label,fk_class,class_label,pk_entity,schema_name,table_name,...,metadata,pk_text,quill_doc,string,fk_text,fk_entity_version,id_for_import,id_for_import_txt,fk_publication_status,fk_license
738,884918,884929,23399767,884031,Gedanke zur Politik,635,Tag,24668649,data,chunk,...,NaN,23399775,"{""ops"": [{""insert"": ""N"", ""attributes"": {""chari...",Nationen,23399767,2,NaN,NaN,NaN,NaN
739,884918,884929,23399767,885073,Herrn Hofrath Schlossers drittes Schreiben übe...,219,Serially Produced Source,24668650,data,chunk,...,NaN,23399779,"{""ops"": [{""insert"": ""E"", ""attributes"": {""chari...",Ephemerid d M J 12 Stück,23399767,2,NaN,NaN,NaN,NaN
740,884918,884929,23399767,885084,Johann Georg Schlosser,21,Person,24668651,data,chunk,...,NaN,23399781,"{""ops"": [{""insert"": ""S"", ""attributes"": {""chari...",Schloßer,23399767,2,NaN,NaN,NaN,NaN


In [24]:
### Éliminer espaces en trop et charactère unicode – espace non sécable
chunks['entity_label'] = chunks['entity_label'].apply(lambda x : str(x).strip().replace(u'\xa0', u' '))

## Analyse des fréquences

In [25]:
### Fréquence par manuscrit

chunks.groupby('id_manuscript').size()

id_manuscript
835550     22
835618     45
870956      3
871001     75
871145     79
872613      5
872634     30
872738     13
872829     21
872914     98
874182    347
884918    117
891345    107
891868     90
892262     72
897087    202
897353    180
897614    200
897895     10
897928    243
898485    190
906998    299
907422    199
907703    143
907947    194
908256    246
908597    281
908609    222
908620    261
908630    183
908642    139
dtype: int64

In [26]:
### Liste des entités (et non des mentions)
distinct_chunks = chunks.drop_duplicates(['id_entity','entity_label','fk_class','class_label'])[['id_entity','entity_label','fk_class','class_label']]
distinct_chunks[:3]

,id_entity,entity_label,fk_class,class_label
0,873964,Selbstkritik,635,Tag
1,873957,Freundschaft,635,Tag
2,873838,Unbekannte Person,635,Tag


### Dictionnaire des entités : distinct_chunks_li



In [27]:
distinct_chunks_li = distinct_chunks.to_dict(orient='records')
distinct_chunks_li[0]

{'id_entity': 873964,
 'entity_label': 'Selbstkritik',
 'fk_class': 635,
 'class_label': 'Tag'}

In [28]:
### Effectifs des classes présentes dans les entités
classes = distinct_chunks.groupby('class_label').size()   ##.count()
print(len(classes))
classes

4


class_label
Geographical Place            3
Person                      284
Serially Produced Source    176
Tag                          97
dtype: int64

In [29]:
### Effectif des individus mentionnées par classe
mention_classes = chunks.groupby('class_label').size()   ##.count()
mention_classes

class_label
Geographical Place            12
Person                      1169
Serially Produced Source     261
Tag                         2874
dtype: int64

In [30]:
### Moyenne mention par individu de classe

cla_men = pd.DataFrame(mention_classes).merge(pd.DataFrame(classes), on = 'class_label')
cla_men.columns =['mentions', 'freq']

def func(x):
    return x[0] / x[1]

cla_men['mean'] = cla_men.apply(func, axis = 1)
cla_men

,mentions,freq,mean
class_label,,,
Geographical Place,12,3,4.000000
Person,1169,284,4.116197
Serially Produced Source,261,176,1.482955
Tag,2874,97,29.628866


In [31]:
mention_entities = chunks[chunks['class_label'].str.contains("Pla")].groupby('entity_label').size()
print(len(mention_entities))
mention_entities.sort_values(ascending = False).to_dict()

3


{'Basel CH': 10, 'Binningen CH': 1, 'Neuenburg CH': 1}

In [32]:
mention_entities = chunks[chunks['class_label'].str.contains("Tag")].groupby('entity_label').size()
print(len(mention_entities))
mention_entities.sort_values(ascending = False).to_dict()

97


{'Religion': 167,
 'Unbekannte Person': 159,
 'Altersgebrechen': 123,
 'Besuch': 120,
 'Gedanken über den Tod': 120,
 'Alter(n)': 111,
 'Gedanken zur Gesellschaft': 92,
 'Schmerz/Krankheit Familie': 91,
 'Ortserwähnung': 91,
 'Selbstreflexion': 89,
 'Ereignis': 79,
 'Gedanken zur Erziehung': 77,
 'Rückblick': 69,
 'Personenbeschreibung': 67,
 'Kantonstrennung': 63,
 'Mädchenschule?': 63,
 'Geschäfte': 63,
 'Trübe Stimmung': 61,
 'Heiterkeit': 53,
 'Selbstkritik': 53,
 'Selbstzeugnis schreiben': 49,
 'Über das weibliche Geschlecht': 49,
 'Brief': 45,
 'Philosophie': 44,
 'Trennung': 40,
 'Ausbildung': 38,
 'Stadt-Land': 37,
 'Krankheit': 34,
 'Tätigkeiten': 34,
 'Gut St. Apollinaris bei Michelbach': 34,
 'Wetter': 33,
 'Sorgen': 30,
 'Bauvorhaben': 29,
 'Trauer': 28,
 'Umzug': 28,
 'Krankheit anderer Personen': 25,
 'Gedanke zur Politik': 25,
 'Reisen': 25,
 'Gedanken zur Ehe': 24,
 'Unnütz/Untätig': 24,
 'Tod': 22,
 'Freundschaft': 22,
 'Ratschlag an die Kinder': 20,
 'Alter(n) anderer

In [33]:
[r['id_entity'] for i,r in distinct_chunks.iterrows()][10:15]

[874019, 910645, 874029, 835869, 835771]

# TEI document production

## Header

In [34]:
for h in hefte_li[:1]:
    print(h['id_manuscript'], h['title_manuscript'])

835549 Heft 1 (1795)


In [63]:
### Attention il s'agit de la valeur id_expression et non id_manuscrit
tei_xml_id = 884918

In [64]:
### Liste des entités (et non des mentions)
distinct_chunks = chunks[chunks['id_manuscript'] == tei_xml_id]\
      .drop_duplicates(['id_entity','entity_label','fk_class','class_label'])[['id_entity','entity_label','fk_class','class_label']]
len(distinct_chunks), distinct_chunks[-3:]

(58,
      id_entity                                       entity_label  fk_class  \
 850     885453                  August Heinrich Julius Lafontaine        21   
 851     885469  Lafontaine, August: Der Rheinfall, in: Minerva...       219   
 853     874045                             Gedanken zur Erziehung       635   
 
                   class_label  
 850                    Person  
 851  Serially Produced Source  
 853                       Tag  )

### Fonction qui prépare la partie listes du header



In [37]:
distinct_chunks_li = distinct_chunks.to_dict(orient='records')
distinct_chunks_li[0]

{'id_entity': 884031,
 'entity_label': 'Gedanke zur Politik',
 'fk_class': 635,
 'class_label': 'Tag'}

In [38]:
def tei_header_lists(chunks, id_expression):
    
    header_result = []
    persons = ''
    objects = ''
    places = ''

    distinct_chunks = chunks[chunks['id_manuscript'] == id_expression]\
        .drop_duplicates(['id_entity','entity_label','fk_class','class_label'])\
        [['id_entity','entity_label','fk_class','class_label']]
        
    
    
    for i,r in distinct_chunks.iterrows():
        if r['fk_class'] == 21:
            hr = '<person xml:id="i' + str(r['id_entity']) + '"><persName>' + \
            r['entity_label'] + '</persName></person>'
            persons += hr
            
        elif r['fk_class'] == 363 :
            hr = '<place xml:id="i' + str(r['id_entity']) + '"><placeName>' + \
            r['entity_label'] + '</placeName></place>'
            places += hr

        else:
            if r['fk_class'] == 635:
                ty = 'tag'
            elif r['fk_class'] == 219:
                ty = 'book_etc'
            else :
                pass    
            hr = '<object xml:id="i' + str(r['id_entity']) + '" type="' + ty + '"><objectIdentifier><objectName>' + \
            r['entity_label'] + '</objectName></objectIdentifier></object>'
            objects += hr
            

    lper = ''
    if len(persons) > 0:
        lper = '<listPerson>' + persons + '</listPerson>'

    lob = ''
    if len(objects) > 0:
        lob = '<listObject>' + objects + '</listObject>'
        
    lpla = ''    
    if len(places) > 0:
        lpla = '<listPlace>' + places + '</listPlace>'
        
    header_result = lper + lob + lpla
    
    return header_result

In [63]:
### Attention il s'agit de la valeur id_expression et non id_manuscrit
id_expression = 884918

In [65]:
tei_header_lists(chunks, id_expression)

'<listPerson><person xml:id="i885084"><persName>Johann Georg Schlosser</persName></person><person xml:id="i867825"><persName>Isaak Iselin-Forcart</persName></person><person xml:id="i885100"><persName>Karl Friedrich Bahrdt</persName></person><person xml:id="i870097"><persName>Sokrates</persName></person><person xml:id="i835771"><persName>Dietrich Preiswerk-Bischoff</persName></person><person xml:id="i836927"><persName>Dietrich Preiswerk-Bischoff</persName></person><person xml:id="i867913"><persName>Maria Helena Hoffmann-Preiswerk</persName></person><person xml:id="i868044"><persName>Sophia Geigy-Preiswerk</persName></person><person xml:id="i869319"><persName>Philipp Emanuel von Fellenberg-Tscharner</persName></person><person xml:id="i867877"><persName>Louise Köchlin</persName></person><person xml:id="i869303"><persName>Margaretha von Fellenberg-Tscharner</persName></person><person xml:id="i885212"><persName>Elisabeth Charlotte Furer-Fellenberg</persName></person><person xml:id="i885225"

## Principe

* Pour chaque caractère on vérifie s'il est dans une balise sémantique, puis on applique un traitement spécifique
* Les balises sont fermées correctement mais si l'emboitement n'est pas correct selon l'arboréscence XML le résultat sera XML valide mais non correct par rapport au sens du codage original

In [40]:
### Fonction qui retourne l'intersection entre deux listes, 
#  i.e. les éléments communs
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

### Fonction qui retourne la différence entre deux listes, 
#  i.e. les éléments qui sont dans l'une mais pas dans l'autre, et réciproquement
def difference(lst1, lst2):
    lst3 = list(set(lst1) ^ set(lst2))
    return lst3

In [41]:
difference([5,1,7,9,6], [3,6,9,8,23]),intersection([5,1,7,9,6], [3,6,9,8,23])

([1, 3, 7, 5, 23, 8], [9, 6])

### Fonction qui prépare les divs

In [70]:
p = re.compile('(\[)(.*)(\])')

In [73]:
p1 = re.compile('\[.*\]')

In [84]:
p2 = re.compile('\[\d+\]')

In [93]:
d = """
<p>[23. November 1795]</p>
<p>[3]</p>
["]
alpha beta
"""

In [85]:
r = p2.findall(d)
r

['[3]']

In [94]:
p3 = re.compile('(<p>\[)(\d+)(\]</p>)')

print(p3.findall(d))

r = p3.findall(d)


for i in r:
    print(i.index)
    rep = '<pb n="' + r[0][1]+  '"/>'
    d = d.replace(''.join(i), rep)
    
print('-----\n',d)    

[('<p>[', '3', ']</p>')]
<built-in method index of tuple object at 0x7fd59af52fc0>
-----
 
<p>[23. November 1795]</p>
<pb n="3"/>
["]
alpha beta



In [82]:
p1 = re.compile('<p>\[.{5,25}\]</p>')

r = p1.findall(d)

for i in r:
    print(i)
    d = d.replace(i, '')
    
print(d)    



<pb n="3"/>
["]
alpha beta



'c'

In [151]:
def divs_text(tei_xml_id, texts, chunks):
    
    divs = ''
    
    ### Sélection des sections de ce manuscrit
    div_texts_li = texts[texts['id_manuscript'] == tei_xml_id].to_dict(orient='records')
    print(len(div_texts_li))

    
    ### Boucle de traitement de chaque section, un texte = 't'
    for t in div_texts_li:
        
        output_text = ''
        
        sect_title = '<head><supplied resp="#editor">'+ t['section_type'].strip(' \'') +'<date> ' + t['title_section'] + ' </date></supplied></head>'
        

        
        section_id = t['id_section']
        text_id = t['id_text']
        
        data = json.loads(t['quill_doc'])
        insert_chars = data['ops']
        
        chun_li = chunks[chunks['id_text'] == t['id_text']].to_dict(orient='records')
        new_chunks = [[i['id_entity'], i['entity_label'], i['fk_class'], i['class_label'], \
                       [i[0] for i in[  (list(e['attributes'].values())) \
                                    for e in json.loads(i['quill_doc'])['ops']]]] \
                                   for i in chun_li]


        
        output = []
        tags_l = []

        len_string = 0
        chars_l = []

        output.append('<p>')

        for l in insert_chars:   #[206:400]:

            pre_char = ''
            post_char =''
            local_tags_l = []
            chars_l.append(l['insert'])

            try:
                a = l['attributes']['charid']
                # print('–––\n', a)

                if a in ['\xa0']:
                    pass
                else:
                    for c in new_chunks:

                        if c[2] == 219:
                            ty = 'book_etc'
                        elif c[2] == 21:
                            ty = 'person'
                        elif c[2] == 363:
                            ty = 'place'    
                        else :
                            ty = 'object'
                            
                            
                        ### si un charactère est dans un chunk et pas encore dans la liste des chuncks
                        # ajouter à la liste et ajouter ouverture élém. 'rs'
                        if a in c[4] and c[0] not in tags_l: 
                            #  print(a + ' : ' + str(c[0])) 
                            pre_char = pre_char + '<rs ref="#i' + str(c[0]) + '" type="'+ ty +'">'
                            tags_l.append(c[0])
                            local_tags_l.append(c[0])
                            #  print('new:', tags_l)
                        elif a in c[4] and c[0] in tags_l and c[0] not in local_tags_l:
                            local_tags_l.append(c[0])
                            pass
                        else:
                            pass

                    ## list difference   
                    diff_l = list(set(tags_l) ^ set(local_tags_l))
                    #  print(diff_l)
                    post_char = '</rs>' * len(diff_l)

                    ## list intersection
                    tags_l = intersection(tags_l, local_tags_l)
                    #  print(tags_l)

                    ### Traiter les tirets
                    
                    ### Remplacer par entités HTML
                    if l['insert'] == '<' :
                        char = '&lt;'
                    elif l['insert'] == '>' : 
                        char = '&gt;'
                    elif l['insert'] == '&' : 
                        char = '&amp;'    
                    else:
                        char = l['insert']

                    output.append(pre_char + post_char + char)
                    len_string += 1
                            



            except:
                # print('b', l['attributes']['blockid'])
                if len_string > 4:
                    output.append('<lb/>')  #  l['insert']
                    len_string = 0
                else:
                    if len(tags_l) > 0:
                        output.append('</rs>' * len(tags_l))
                        tags_l = []
                    output.append('</p><p>')
                    len_string = 0
       

        if len(tags_l) > 0:
            output.append('</rs>' * len(tags_l))
            tags_l = []
        output.append('</p>')    
        
        output_text = '<div xml:id="d' + str(text_id) + '" corresp="d' + str(section_id) + '" >' \
                    + sect_title + (''.join(output).replace('<lb/></p>', '</p>')) + '</div>'
        
       

        divs += output_text.replace('<p></p>','')\
                    .replace('{', '<del>').replace('}', '</del>')\
                    .replace('&lt;', '<add>').replace('&gt;', '</add>')
        
        
        p2 = re.compile('(<p>\[)(\d+)(\]</p>)')
        r = p2.findall(divs)
        for i in r:
            rep = '<pb n="' + i[1]+  '"/>'
            divs = divs.replace(''.join(i), rep)    

            
        p1 = re.compile('<p>\[.{5,35}\]</p>')
        r = p1.findall(divs)
        for i in r:
            divs = divs.replace(i, '')            
            
        
        try:
            x_doc = etree.fromstring(output_text)
            result_list.append('[' + str(tei_xml_id) + '] ' + str(text_id) + ' : ok !')
            doc_list.append(tei_xml_id)
        except Exception as e:
            result_list.append(str(tei_xml_id) + '–' + str(section_id) + t['title_section'] + " : error !")
            result_list.append(e)
                                              
    return divs
                                              

In [153]:
from datetime import datetime
 
dt_begin = datetime.now()
 

doc_list = []
result_list = []
for h in hefte_li:# [4:5]:  # [:1]
    
    tei_document = ''
    tei_xml_id = h['id_manuscript']
    id_expression = h['id_expression']

    
    ##  <?xml version="1.0" encoding="UTF-8"?>
    
    tei_document = """<?xml version="1.0"?>
    <?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>    
    <TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="d""" + str(tei_xml_id) + """">
       <teiHeader>
          <fileDesc>
             <titleStmt>
                <title>Tagebuch """ + h['title_manuscript'] +  """</title>
                <author>Anna Maria Preiswerk-Iselin</author>
                <editor>Prof. Fr. Claudia Opitz-Belakhal</editor>
                <editor>Anna Cristina Münch</editor>
                <editor>Selina Bentsch</editor>
                <editor>Esther Baur</editor>
                <editor>Andreas Berger</editor>
             </titleStmt>

             <publicationStmt>
                <publisher xml:id="editor">Tagebücher Anna Maria Preiswerk-Iselin Projekt</publisher>
                <pubPlace>Basel</pubPlace>
                <date when="2022-03">March 2022</date>
               <availability>
                   <licence notBefore="2022-03-01" target="http://creativecommons.org/licenses/by-sa/4.0/">
                      <p>[...]</p>
                      <p>This work is licensed under a Creative Commons Attribution-ShareAlike 4.0
                         International License.</p>
                      <p> You are free to: <list>
                         <item> Share — copy and redistribute the material in any medium or
                            format</item>
                         <item> Adapt — remix, transform, and build upon the material</item>
                      </list>
                      </p>
                      <p> Under the following terms: <list>
                         <item> Attribution — You must give appropriate credit, provide a link to the
                            license, and indicate if changes were made. You may do so in any
                            reasonable manner, but not in any way that suggests the licensor endorses
                            you or your use.</item>
                         <item> ShareAlike — If you remix, transform, or build upon the material, you
                            must distribute your contributions under the same license as the
                            original.</item>
                         <item> No additional restrictions — You may not apply legal terms or
                            technological measures that legally restrict others from doing anything
                            the license permits.</item>
                      </list>
                      </p>
                      <p>The licence was added on March first, 2022.</p>
                   </licence>
                </availability>
             </publicationStmt>
             <seriesStmt>
                <title>Tagebücher Anna Maria Preiswerk-Iselin</title>
                <respStmt>
                   <resp>Transkription</resp>
                   <resp xml:lang="en">Transcriber</resp>
                   <persName>Esther Baur</persName>
                   <orgName>Staatsarchiv Basel</orgName>
                </respStmt>
                <respStmt>
                   <resp>Transkription</resp>
                   <resp xml:lang="en">Transcriber</resp>
                   <persName>Andreas Berger</persName>
                </respStmt>                
                <respStmt>
                   <resp>Projektleitung</resp>
                   <resp xml:lang="en">Principal investigator</resp>
                   <persName>Prof. Fr. Claudia Opitz-Belakhal</persName>
                   <orgName>Universität Basel</orgName>
                </respStmt>
                <respStmt>
                   <resp>Wissenschaftliche Mitarbeiterin, Doktorandin</resp>
                   <resp xml:lang="en">Scientific collaborator, PhD student</resp>
                   <persName>Selina Bentsch</persName>
                   <orgName>Universität Basel</orgName>
                </respStmt>
                <respStmt>
                   <resp>Projektmitarbeiterin</resp>
                   <resp xml:lang="en">Project collaborator</resp>
                   <persName>Anna Cristina Münch</persName>
                   <orgName>Universität Basel</orgName>
                </respStmt>
                <biblScope unit="volume">Heft 31</biblScope>
             </seriesStmt>
             <sourceDesc>
                <msDesc>
                   <msIdentifier>
                      <settlement>Basel</settlement>
                      <repository>Staatsarchiv</repository>
                      <idno>StABS PA 511 304-03-04</idno>
                      <msName>""" + h['title_manuscript'] +  """ - Anna Maria Preiswerk-Iselin </msName>
                   </msIdentifier>
                </msDesc>
         </sourceDesc>
      </fileDesc>
      <profileDesc>
         <textClass>
            <catRef scheme="#genre" target="#journal"/>
         </textClass>
         <langUsage>
            <language ident="de">Deutsch</language>
         </langUsage>
      </profileDesc>
   </teiHeader>
       <text><body>""" + divs_text(tei_xml_id, texts, chunks) + """
       </body></text>
       <standOff>
       """  + tei_header_lists(chunks, id_expression) + """
       </standOff>
   </TEI>
    """
    
    try:
        x_doc = etree.fromstring(tei_document) #  parse
        result_list.append(str(tei_xml_id) + ' : -> file ok !')
        doc_list.append(tei_xml_id)
    except Exception as e:
        result_list.append(str(tei_xml_id) + " : -> file error !")
        result_list.append(str(e))
        
        
    file_name = 'tei_files/text_i' + str(tei_xml_id) + '.xml'
    
    with open(file_name, 'w') as f:
        f.write(tei_document)

dt_end = datetime.now()        
        
logs = 'texte/logs.txt'
logs_content = '\n'.join(result_list)
with open(logs, 'w') as f:

    f.write('--- start: ' + dt_begin.strftime("%Y%m%d_%H%M%S") + '\n')

    f.write(logs_content)
    
    f.write('\n--- end: ' + dt_end.strftime("%Y%m%d_%H%M%S"))


4
8
1
9
19
1
6
4
5
19
65
18
27
23
14
28
30
31
3
31
28
25
45
30
26
33
36
49
49
44
35
37


In [100]:
len(doc_list)

0

## Question importante

Implémenter cette fonction en PL/pgSQL ou Python dans PostgreSQL pour produire le texte de sortie directement sur le serveur, dans une fonction PL/pgSQL par exemple, renvoyant directement le texte XML/TEI